# Multiple Linear Regression Lab : House Prices

### Overview
Estimate house prices using MLR

### Builds on
None

### Run time
approx. 20 minutes

### Notes


## Step 1 - Load Data

In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import os

data_location = "../data/house-prices/house-sales-full.csv"
if not os.path.exists (data_location):
    data_location = 'https://elephantscale-public.s3.amazonaws.com/data/house-prices/house-sales-full.csv'
print('data_location:', data_location)

house_prices = pd.read_csv(data_location)
house_prices

data_location: ../data/house-prices/house-sales-full.csv


,DocumentID,Date,SalePrice,PropertyID,PropertyType,ym,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,...,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
0,1,9/16/14,280000,1000102,Multiplex,9/1/14,405100,0.93,"300,805.00",2,...,3.00,6,7,1991,0,0,70000,229000,98002,False
1,2,6/16/06,1000000,1200013,Single Family,6/1/06,404400,0.93,"1,076,162.00",1,...,3.75,4,10,2005,0,0,203000,590000,98166,True
2,3,1/29/07,745000,1200019,Single Family,1/1/07,425600,0.98,"761,805.00",1,...,1.75,4,8,1947,0,0,183000,275000,98166,False
3,4,2/25/08,425000,2800016,Single Family,2/1/08,418400,0.96,"442,065.00",1,...,3.75,5,7,1966,0,0,104000,229000,98168,False
4,5,3/29/13,240000,2800024,Single Family,3/1/13,351600,0.81,"297,065.00",1,...,1.75,4,7,1948,0,0,104000,205000,98168,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27058,27059,10/31/12,374000,9895000030,Townhouse,10/1/12,332400,0.76,"489,665.00",1,...,1.75,2,9,2011,0,0,147000,269000,-1,False
27059,27060,12/7/12,374000,9895000040,Townhouse,12/1/12,339400,0.78,"479,566.00",1,...,1.75,2,9,2011,0,0,147000,269000,-1,True
27060,27061,7/9/12,165000,9899200010,Single Family,7/1/12,325300,0.75,"220,744.00",1,...,1.00,4,6,1971,0,0,92000,130000,98055,False
27061,27062,5/26/06,315000,9900000355,Single Family,5/1/06,400600,0.92,"342,207.00",1,...,2.00,3,7,1939,0,0,103000,212000,98166,False


## Step 2 - Explore Data (EDA)
EDA is a great way to get a sense of the data.  

Try to find answers to the following questions, by looking at the output of `describe` below

- How many 'max' bedrooms do we have? :-)
- Find min/max of 'SalePrice'
- Do you think we have outliers in data

In [21]:
## describe
house_prices.describe()

,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode
count,"27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00"
mean,"13,532.00","511,626.20","4,680,324,882.08","390,750.58",0.90,"570,918.76",1.02,"10,997.68","2,122.96",275.30,2.26,3.38,7.78,"1,977.09",86.73,0.21,"213,820.07","317,211.67","82,223.04"
std,"7,812.56","342,821.17","2,896,350,979.15","37,024.46",0.09,"380,236.63",0.15,"28,110.66",939.84,428.71,0.77,0.90,1.18,30.92,407.32,0.55,"177,213.41","234,038.34","36,106.67"
min,1.00,"3,000.00","1,000,102.00","311,600.00",0.72,"3,368.00",1.00,494.00,370.00,0.00,0.00,0.00,3.00,"1,900.00",0.00,0.00,0.00,0.00,-1.00
25%,"6,766.50","329,000.00","2,213,000,057.50","357,100.00",0.82,"366,918.50",1.00,"4,257.50","1,440.00",0.00,1.75,3.00,7.00,"1,954.00",0.00,0.00,"105,000.00","183,000.00","98,019.00"
50%,"13,532.00","425,000.00","3,972,900,140.00","400,600.00",0.92,"475,664.00",1.00,"6,636.00","1,940.00",0.00,2.50,3.00,8.00,"1,986.00",0.00,0.00,"172,000.00","261,000.00","98,053.00"
75%,"20,297.50","590,000.00","7,504,001,385.00","421,200.00",0.97,"655,061.00",1.00,"9,450.00","2,610.00",510.00,2.50,4.00,8.00,"2,006.00",0.00,0.00,"258,000.00","380,000.00","98,115.00"
max,"27,063.00","11,000,000.00","9,906,000,035.00","435,200.00",1.00,"11,644,855.00",5.00,"1,024,068.00","10,740.00","3,500.00",8.00,33.00,13.00,"2,016.00","2,016.00",3.00,"5,538,000.00","5,772,000.00","98,354.00"


In [22]:
## any correlation
house_prices.corr()

,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
DocumentID,1.00,-0.02,0.99,-0.02,-0.02,-0.02,-0.02,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.01,-0.01,0.05
SalePrice,-0.02,1.00,-0.02,0.12,0.12,0.98,0.02,0.14,0.68,0.29,0.52,0.32,0.66,0.08,0.09,-0.01,0.79,0.81,-0.02,0.04
PropertyID,0.99,-0.02,1.00,-0.02,-0.02,-0.02,-0.01,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.02,-0.01,0.05
zhvi_px,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
zhvi_idx,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
AdjSalePrice,-0.02,0.98,-0.02,-0.03,-0.03,1.00,0.02,0.14,0.69,0.29,0.52,0.32,0.67,0.08,0.09,-0.02,0.80,0.82,-0.03,0.04
NbrLivingUnits,-0.02,0.02,-0.01,0.02,0.02,0.02,1.00,-0.00,0.05,0.06,0.10,0.16,-0.05,-0.11,0.01,0.08,0.03,-0.00,0.04,-0.05
SqFtLot,-0.11,0.14,-0.11,-0.00,-0.00,0.14,-0.00,1.00,0.21,0.05,0.10,0.08,0.14,0.03,0.01,-0.01,0.08,0.11,0.06,-0.09
SqFtTotLiving,-0.02,0.68,-0.02,-0.01,-0.01,0.69,0.05,0.21,1.00,0.39,0.75,0.62,0.76,0.30,0.05,-0.07,0.47,0.76,-0.10,0.11
SqFtFinBasement,-0.03,0.29,-0.03,0.02,0.02,0.29,0.06,0.05,0.39,1.00,0.26,0.30,0.14,-0.22,0.11,0.05,0.29,0.28,0.10,-0.14


## Step 3 - Shape Data
Wow! That's a lot of columns.  Maybe we should foucs on just a few of them.

**=> Select only "SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"**

In [23]:
## TODO : select columns
input_columns= ['Bedrooms', 'Bathrooms', 'SqFtTotLiving', 'SqFtLot']
label_column = ['SalePrice']

house_prices2 = house_prices[input_columns + label_column]
house_prices2

,Bedrooms,Bathrooms,SqFtTotLiving,SqFtLot,SalePrice
0,6,3.00,2400,9373,280000
1,4,3.75,3764,20156,1000000
2,4,1.75,2060,26036,745000
3,5,3.75,3200,8618,425000
4,4,1.75,1720,8620,240000
...,...,...,...,...,...
27058,2,1.75,1410,1161,374000
27059,2,1.75,1410,1005,374000
27060,4,1.00,1070,11170,165000
27061,3,2.00,1345,6223,315000


In [24]:
x = house_prices2[input_columns]
y = house_prices2[label_column]

print ("x.shape = ", x.shape)
print ("y.shape = ", y.shape)

x.shape =  (27063, 4)
y.shape =  (27063, 1)


## Step 4 : Run Regression

In [25]:
from sklearn.linear_model import LinearRegression

model = LinearRegression ().fit(x,y)
model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## Step 5 : Predict

In [26]:
predictions = model.predict(x)
predictions

array([[429991.83],
       [950801.59],
       [444361.66],
       ...,
       [158195.01],
       [316458.05],
       [154251.23]])

In [27]:
pd.options.display.float_format = '{:,.2f}'.format

a = house_prices2[input_columns  + label_column]
a['predicted_price'] = predictions
a

,Bedrooms,Bathrooms,SqFtTotLiving,SqFtLot,SalePrice,predicted_price
0,6,3.00,2400,9373,280000,"429,991.83"
1,4,3.75,3764,20156,1000000,"950,801.59"
2,4,1.75,2060,26036,745000,"444,361.66"
3,5,3.75,3200,8618,425000,"729,743.95"
4,4,1.75,1720,8620,240000,"351,192.04"
...,...,...,...,...,...,...
27058,2,1.75,1410,1161,374000,"396,038.84"
27059,2,1.75,1410,1005,374000,"396,050.47"
27060,4,1.00,1070,11170,165000,"158,195.01"
27061,3,2.00,1345,6223,315000,"316,458.05"


## Step 6 : Evaluate

**Q==> Are any coefficients close to zero?  What does that mean?**

**Q==> Also inspect R2 value.  Is it decent?**

In [28]:
## coefficients
import numpy as np
np.set_printoptions(precision=2, suppress=True)

print ("coefficients: " , model.coef_)
print ("intercept : ", model.intercept_)

coefficients:  [[-65211.74  16274.19    277.85     -0.07]]
intercept :  [106303.31]


In [29]:
## print each feature and it's coefficients
coef = pd.DataFrame({"input_column" : input_columns,  
                     "coefficient": model.coef_[0]})
coef

,input_column,coefficient
0,Bedrooms,"-65,211.74"
1,Bathrooms,"16,274.19"
2,SqFtTotLiving,277.85
3,SqFtLot,-0.07


In [30]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

print ("R2 : " , r2_score(y, predictions))
print ("MSE : ", mean_squared_error(y, predictions))
print ("RMSE : ", sqrt(mean_squared_error(y, predictions)) )

R2 :  0.483482252362197
MSE :  60702204448.745056
RMSE :  246378.1736452015


## Step 7 : Predict on New Data

In [31]:
# Make a Pandas dataframe
new_data = pd.DataFrame({'Bedrooms' : [5,3,2],
                         'Bathrooms' : [3,2,1.5],
                         'SqFtTotLiving' : [4400, 1800, 1550],
                         'SqFtLot' : [10000, 5000, 4000]
             })
new_data

,Bedrooms,Bathrooms,SqFtTotLiving,SqFtLot
0,5,3.00,4400,10000
1,3,2.00,1800,5000
2,2,1.50,1550,4000


In [32]:
new_prediction = model.predict(new_data)
new_prediction

array([[1050852.9 ],
       [ 442970.11],
       [ 430657.31]])

In [33]:
pd.options.display.float_format = '{:,.2f}'.format

new_data['predicted_price'] = new_prediction
new_data

,Bedrooms,Bathrooms,SqFtTotLiving,SqFtLot,predicted_price
0,5,3.00,4400,10000,"1,050,852.90"
1,3,2.00,1800,5000,"442,970.11"
2,2,1.50,1550,4000,"430,657.31"


## Step 8 : Imporove Model Performance
Now that we have done a 'end-to-end' implementation in Regression.  
How ever our accuracy isn't all that great!  

**Q ==> What can we do to improve our model?**

One option is to choose better input columns.  
in Step-3, add more input columns.
For example you can add 'LandVal' to input columns as follows

```python
input_columns= ['Bedrooms', 'Bathrooms', 'SqFtTotLiving', 'SqFtLot', 'LandVal']
```

And then run the notebook again, and see if the accuracy goes up!

### Class Competition

Let's see how high you can bring up the model accuracy, with fewest columns as possible.